# Random Controller Tutorial
This tutorial will explain how to use both the discrete and continuous version of the random action controller.

In [ ]:
import random
from abc import ABC, abstractmethod
from typing import List, Tuple, Union

import numpy as np

from psipy.rl.plant import Action, State
from psipy.rl.control.controller import DiscreteRandomActionController, ContinuousRandomActionController

## Actions
Actions come associated with a `dtype`.  This explains what type of action space they will have, either 'discrete' or 'continuous'.  Only the proper action types (discrete or continuous) can go into their respective controllers.

Note `legal_values`: this is a tuple where each item corresponds to the respective channel.  These define the lower and upper (both inclusive) limits of that channel for continuous actions, and the actual action space for discrete actions.  For example, in the `DiscreteAction`, `channel1` has can take values in the range [0, 99].

In [ ]:
class DiscreteAction(Action):
    dtype = "discrete"
    channels = (
        "channel1",
        "channel2"
    )
    legal_values = (range(0, 100), range(-10, 10))  # Actual values to sample from
    
class ContinuousAction(Action):
    dtype='continuous'
    channels = (
        "channel1",
        "channel2",
        "channel3"
    )
    legal_values = ((0,100), (-10, 10), (-10, -5))  # [Lower, Upper]

class CustomState(State):
    _channels = (
        "state1",
        "state2"
    )

## Random Action Controllers
These controllers sample randomly from the action spaces as defined like above.  They take in the state and action types that they will be manipulating, as well as a cycle manager to record cycle times.  A delay can also be applied, in order to make the actions not jitter between every state.  The `delay` parameter controlls how many states the controller must see before it computes another action.

### Invalid controller/action combination
Here we try to put a discrete action into a continuous controller.  You can not sample continuously from a discrete space, so this throws an error.

The same occurs when trying to put a continuous action into a discrete controller.  It is unknown how you would like to discretize the space, and so an error is thrown.  In this case, it would be best to create a discrete action with the discretization you desire.

In [ ]:
try:
    breaks = ContinuousRandomActionController(CustomState.channels(), 
                                              DiscreteAction)
except ValueError as e:
    print(e)

## Using the controllers

### Fake data
Here we generate fake state data since actions are a function of states.

In [ ]:
data = np.ones(len(CustomState.channels()))
state = CustomState(data)

### Continuous controller
Here we make a continuous controller and get an action a few times.  You can see that the ranges being sampled correspond to the `legal_values` class variable.

In [ ]:
crc = ContinuousRandomActionController(CustomState.channels(),
                                       ContinuousAction)
print(crc.get_action(state))
print(crc.get_action(state))
print(crc.get_action(state))
print(crc.get_action(state))
print(crc.get_action(state))

### Discrete controller with discrete action
Here we create a discrete action controller and generate a few actions.  Note also that the values are sampled from the `legal_values` class variable.

In [ ]:
drc1 = DiscreteRandomActionController(CustomState.channels(),
                                      DiscreteAction)
print(drc1.get_action(state))
print(drc1.get_action(state))
print(drc1.get_action(state))
print(drc1.get_action(state))
print(drc1.get_action(state))

### Discrete Controller with delayed discrete action
Here we create a discrete action controller and specify a delay of 5.  Notice how the state only changes every 5 states/`get_action` calls.

In [ ]:
drc4 = DiscreteRandomActionController(CustomState.channels(),
                                      DiscreteAction,
                                      num_repeat=5)
print(drc4.get_action(state))
print(drc4.get_action(state))
print(drc4.get_action(state))
print(drc4.get_action(state))
print(drc4.get_action(state))
print("Change here--")
print(drc4.get_action(state))
print(drc4.get_action(state))
print(drc4.get_action(state))
print(drc4.get_action(state))
print(drc4.get_action(state))